In [ ]:
import glob
import os

import pandas as pd
import plotly.express as px

In [ ]:
data_dir = "benchmark_results_5090"
os.chdir(data_dir)
os.listdir()

In [ ]:
metrics = ["quantize_dequantize_latency", "quantize_latency", "dequantize_latency"]


def load_df(df_type):
    df_paths = glob.glob("baseline_*.csv" if df_type == "baseline" else "improved_*.csv")
    df = pd.DataFrame()

    for path in df_paths:
        run_id = path.split("_")[-1].split(".")[0]
        run_df = pd.read_csv(path)
        # convert to seconds
        for metric in metrics:
            run_df[metric] = run_df[metric] * 1e-6
        run_df["run_id"] = run_id
        df = pd.concat([df, run_df])

    # add a mean of all runs
    cols = ["quant_type", "dtype", "tensor_shape", "blocksize"]
    avg_run = df.groupby(cols)[metrics].mean().reset_index()
    avg_run["run_id"] = "avg"
    df = pd.concat([df, avg_run])

    df["version"] = df_type
    return df


df_improved = load_df("improved")
df_baseline = load_df("baseline")

df = pd.concat([df_improved, df_baseline], ignore_index=True)
df

In [ ]:
def plot(df, algorithm, dtype, run_id="run1"):
    df = df[df.dtype == dtype]
    df = df[df.run_id == run_id]
    param = algorithm
    fig = px.line(
        df,
        x="blocksize",
        y=param,
        color="version",
        facet_col="tensor_shape",
        facet_row="quant_type",
        title=f"{algorithm} by Block Size [1k iterations total latency per blocksize][{dtype}]",
        labels={param: "", "blocksize": "Block Size", "tensor_shape": "dim", "quant_type": "Quantization"},
    )

    fig.add_annotation(
        text=f"Total {algorithm} Time (s)",
        xref="paper",
        yref="paper",
        x=-0.125,
        y=0.5,
        showarrow=False,
        textangle=-90,
        font=dict(size=14),
    )

    # set discrete x-tick labels using unique blocksize values
    blocksize_values = sorted(df["blocksize"].unique())
    for axis in fig.layout:
        if axis.startswith("xaxis"):
            fig.layout[axis].type = "category"
            fig.layout[axis].tickmode = "array"
            fig.layout[axis].tickvals = blocksize_values
            fig.layout[axis].ticktext = [str(val) for val in blocksize_values]
        # elif axis.startswith("yaxis"):
        # fig.layout[axis].type = "log"

    fig.update_yaxes(matches=None)
    fig.update_xaxes(matches=None)
    fig.update_layout(width=1800, height=500)
    fig.show()


plot(df, "quantize_dequantize_latency", "torch.float32", "avg")

In [ ]:
def plot(df, algorithm, dtype, run_id="run1"):
    param = algorithm
    df = df[df.dtype == dtype]
    df = df[df.run_id == run_id]

    df_baseline = df[df.version == "baseline"].reset_index()
    df_improved = df[df.version == "improved"].reset_index()

    df_improved["diff_pct"] = ((df_baseline[param] - df_improved[param]) / df_baseline[param]) * 100
    fig = px.bar(
        df_improved,
        x="blocksize",
        y="diff_pct",
        color=df_improved["diff_pct"],
        facet_row="tensor_shape",
        facet_col="quant_type",
        # barmode="overlay",
        title=f"{algorithm} - Percentage Improvement from Baseline by Block Size [1000 iterations total][{dtype}][Run={run_id}]",
        labels={"diff_pct": "", "blocksize": "Block Size", "tensor_shape": "dim"},
        color_continuous_scale=[[0, "#d62728"], [0.5, "#ffffff"], [1, "#2ca02c"]],
    )
    fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.5)

    fig.add_annotation(
        text="Percentage difference from baseline (%)",
        xref="paper",
        yref="paper",
        x=-0.065,
        y=0.5,
        showarrow=False,
        textangle=-90,
        font=dict(size=14),
    )

    fig.add_annotation(
        text="Green: Improvement | Red: Degradation",
        xref="paper",
        yref="paper",
        x=0.5,
        y=1.02,
        showarrow=False,
        font=dict(size=12),
    )

    # set discrete x-tick labels using unique blocksize values
    blocksize_values = sorted(df["blocksize"].unique())
    for axis in fig.layout:
        if axis.startswith("xaxis"):
            fig.layout[axis].type = "category"
            fig.layout[axis].tickmode = "array"
            fig.layout[axis].tickvals = blocksize_values
            fig.layout[axis].ticktext = [str(val) for val in blocksize_values]
        # elif axis.startswith("yaxis"):
        #     fig.layout[axis].type = "log"

    # fig.update_yaxes(matches=None)
    fig.update_layout(width=1200, height=800)
    fig.show()


run_id = "run3"
plot(df, "quantize_latency", "torch.float32", run_id)
plot(df, "dequantize_latency", "torch.float32", run_id)
plot(df, "quantize_dequantize_latency", "torch.float32", run_id)

In [ ]:
agg_baseline = (
    df_baseline.drop(columns="version")
    .groupby(["run_id", "dtype", "quant_type", "tensor_shape", "blocksize"])
    .median()
)
agg_improved = (
    df_improved.drop(columns="version")
    .groupby(["run_id", "dtype", "quant_type", "tensor_shape", "blocksize"])
    .median()
)
diff = (agg_baseline - agg_improved) / agg_improved * 100
diff = diff.reset_index()
cols = ["quantize_latency", "dequantize_latency", "quantize_dequantize_latency"]

dtype = "torch.float32"
quant_type = "fp4"
run_id = "run1"

mask = (diff.dtype == dtype) & (diff.quant_type == quant_type) & (diff.run_id == run_id)
diff[mask].drop(columns=["dtype", "quant_type"]).groupby(["tensor_shape", "blocksize"])[cols].median()